In [1]:
!python -c "import monai" || pip install -q "monai-weekly[ignite, tqdm]"

In [2]:
import logging
import numpy as np
import os
from pathlib import Path
import sys
import tempfile
import torch

from monai.apps import MedNISTDataset
from monai.config import print_config
from monai.data import DataLoader
from monai.engines import SupervisedTrainer
from monai.handlers import StatsHandler
from monai.inferers import SimpleInferer
from monai.networks import eval_mode
from monai.networks.nets import densenet121
from monai.transforms import LoadImageD, EnsureChannelFirstD, ScaleIntensityD, Compose

print_config()

MONAI version: 1.2.0
Numpy version: 1.23.5
Pytorch version: 2.0.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: /home/minyoungxi/anaconda3/envs/TorchEnv/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.12
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: 0.21.0
Pillow version: 8.4.0
Tensorboard version: 2.13.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.0
tqdm version: 4.65.0
lmdb version: 1.4.1
psutil version: 5.9.0
pandas version: 2.0.3
einops version: 0.6.1
transformers version: 4.30.2
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [3]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/tmp/tmpv50n2gnm


## Use MONAI transforms to preprocess data - MONAI 변환을 사용하여 데이터 사전 처리

의료 이미지에는 I/O, 전처리 및 확대를 위한 특수한 방법이 필요합니다. 그들은 종종 특정 형식을 따르고 특정 프로토콜로 처리되며 데이터 배열은 종종 고차원입니다.

이 예제에서는 아래에 나열된 세 가지 monai.transform으로 이미지 로딩, 데이터 형식 검증 및 강도 스케일링을 수행하고 다음 단계에서 사용할 준비가 된 파이프라인을 구성합니다.

In [4]:
transform = Compose(
    [
        LoadImageD(keys="image", image_only=True),
        EnsureChannelFirstD(keys="image"),
        ScaleIntensityD(keys="image"),
    ]
)

## MONAI 앱을 사용하여 데이터 세트 준비

MONAI 앱에서 MedNISTDataset을 사용하여 데이터 세트를 지정된 디렉토리에 다운로드하고 monai.transforms 작성에서 전처리 단계를 수행합니다.

MedNIST 데이터 세트는 TCIA, RSNA Bone Age Challenge 및 NIH Chest X-ray 데이터 세트의 여러 세트에서 수집되었습니다.

In [5]:
dataset = MedNISTDataset(root_dir=root_dir, transform=transform, section="training", download=True)

MedNIST.tar.gz: 59.0MB [00:03, 19.4MB/s]                                        

2023-08-16 23:57:02,096 - INFO - Downloaded: /tmp/tmpv50n2gnm/MedNIST.tar.gz


2023-08-16 23:57:02,175 - INFO - Verified 'MedNIST.tar.gz', md5: 0bc7306e7427e00ad1c5526a6677552d.
2023-08-16 23:57:02,176 - INFO - Writing into directory: /tmp/tmpv50n2gnm.


Loading dataset: 100%|██████████████████| 47164/47164 [00:14<00:00, 3342.22it/s]


## 네트워크 및 감독 트레이너 정의

분류 작업을 수행할 수 있는 모델을 훈련하기 위해 ImageNet 데이터 세트에서 성능이 뛰어난 것으로 알려진 DenseNet-121을 사용합니다.

일반적인 지도 학습 워크플로우의 경우 MONAI는 하이퍼 매개변수를 정의하기 위해 SupervisedTrainer를 제공합니다.

In [6]:
torch.cuda.is_available()

True

In [7]:
max_epochs = 5
model = densenet121(spatial_dims=2, in_channels=1, out_channels=6).to("cuda:0")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

trainer = SupervisedTrainer(
    device=torch.device("cuda:0"),
    max_epochs=max_epochs,
    train_data_loader=DataLoader(dataset, batch_size=512, shuffle=True, num_workers=4),
    network=model,
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-5),
    loss_function=torch.nn.CrossEntropyLoss(),
    inferer=SimpleInferer(),
    train_handlers=StatsHandler(),
)

/home/minyoungxi/anaconda3/envs/TorchEnv/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.handlers.stats_handler StatsHandler.__init__:name: Current default value of argument `name=None` has been deprecated since version 1.1. It will be changed to `name=StatsHandler` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


In [8]:
trainer.run() # train

INFO:ignite.engine.engine.SupervisedTrainer:Engine run resuming from iteration 0, epoch 0 until 5 epochs
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 1/93 -- label: 0.0000 loss: 1.9166 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 2/93 -- label: 2.0000 loss: 1.8771 


/home/minyoungxi/anaconda3/envs/TorchEnv/lib/python3.8/site-packages/monai/handlers/stats_handler.py:262: UserWarning: ignoring non-scalar output in StatsHandler, make sure `output_transform(engine.state.output)` returns a scalar or dictionary of key and scalar pairs to avoid this warning. image:<class 'monai.data.meta_tensor.MetaTensor'>
  warnings.warn(
/home/minyoungxi/anaconda3/envs/TorchEnv/lib/python3.8/site-packages/monai/handlers/stats_handler.py:262: UserWarning: ignoring non-scalar output in StatsHandler, make sure `output_transform(engine.state.output)` returns a scalar or dictionary of key and scalar pairs to avoid this warning. pred:<class 'monai.data.meta_tensor.MetaTensor'>
  warnings.warn(


INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 3/93 -- label: 4.0000 loss: 1.8470 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 4/93 -- label: 2.0000 loss: 1.8119 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 5/93 -- label: 5.0000 loss: 1.7749 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 6/93 -- label: 0.0000 loss: 1.7505 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 7/93 -- label: 3.0000 loss: 1.7222 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 8/93 -- label: 1.0000 loss: 1.6748 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 9/93 -- label: 1.0000 loss: 1.6690 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 10/93 -- label: 0.0000 loss: 1.6220 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 11/93 -- label: 0.0000 loss: 1.5929 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 12/93 -- label: 2.0000 loss: 1.5718 
INFO:ignite.engin

INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 86/93 -- label: 4.0000 loss: 0.4406 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 87/93 -- label: 3.0000 loss: 0.4160 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 88/93 -- label: 3.0000 loss: 0.4048 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 89/93 -- label: 2.0000 loss: 0.4093 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 90/93 -- label: 0.0000 loss: 0.4265 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 91/93 -- label: 5.0000 loss: 0.4086 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 92/93 -- label: 4.0000 loss: 0.4198 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/5, Iter: 93/93 -- label: 0.0000 loss: 0.4065 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[1] Complete. Time taken: 00:00:14.573
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 1/93 -- label: 1.0000 loss: 0.3799 
INFO:ignite.engine.eng

INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 75/93 -- label: 1.0000 loss: 0.1551 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 76/93 -- label: 3.0000 loss: 0.1844 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 77/93 -- label: 5.0000 loss: 0.1689 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 78/93 -- label: 1.0000 loss: 0.1669 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 79/93 -- label: 1.0000 loss: 0.1862 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 80/93 -- label: 0.0000 loss: 0.1507 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 81/93 -- label: 0.0000 loss: 0.1557 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 82/93 -- label: 0.0000 loss: 0.1510 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 83/93 -- label: 0.0000 loss: 0.1508 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 2/5, Iter: 84/93 -- label: 5.0000 loss: 0.1502 
INFO:ignit

INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 3/5, Iter: 64/93 -- label: 1.0000 loss: 0.0932 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 3/5, Iter: 65/93 -- label: 4.0000 loss: 0.0853 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 3/5, Iter: 66/93 -- label: 5.0000 loss: 0.0956 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 3/5, Iter: 67/93 -- label: 0.0000 loss: 0.0838 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 3/5, Iter: 68/93 -- label: 0.0000 loss: 0.0888 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 3/5, Iter: 69/93 -- label: 4.0000 loss: 0.0827 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 3/5, Iter: 70/93 -- label: 2.0000 loss: 0.0936 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 3/5, Iter: 71/93 -- label: 4.0000 loss: 0.0926 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 3/5, Iter: 72/93 -- label: 4.0000 loss: 0.0800 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 3/5, Iter: 73/93 -- label: 3.0000 loss: 0.0780 
INFO:ignit

INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/5, Iter: 53/93 -- label: 3.0000 loss: 0.0475 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/5, Iter: 54/93 -- label: 4.0000 loss: 0.0569 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/5, Iter: 55/93 -- label: 0.0000 loss: 0.0477 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/5, Iter: 56/93 -- label: 4.0000 loss: 0.0701 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/5, Iter: 57/93 -- label: 2.0000 loss: 0.0487 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/5, Iter: 58/93 -- label: 3.0000 loss: 0.0599 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/5, Iter: 59/93 -- label: 0.0000 loss: 0.0426 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/5, Iter: 60/93 -- label: 0.0000 loss: 0.0481 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/5, Iter: 61/93 -- label: 3.0000 loss: 0.0521 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/5, Iter: 62/93 -- label: 0.0000 loss: 0.0538 
INFO:ignit

INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 5/5, Iter: 42/93 -- label: 1.0000 loss: 0.0497 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 5/5, Iter: 43/93 -- label: 5.0000 loss: 0.0461 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 5/5, Iter: 44/93 -- label: 3.0000 loss: 0.0358 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 5/5, Iter: 45/93 -- label: 4.0000 loss: 0.0310 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 5/5, Iter: 46/93 -- label: 2.0000 loss: 0.0463 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 5/5, Iter: 47/93 -- label: 4.0000 loss: 0.0417 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 5/5, Iter: 48/93 -- label: 1.0000 loss: 0.0394 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 5/5, Iter: 49/93 -- label: 0.0000 loss: 0.0360 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 5/5, Iter: 50/93 -- label: 2.0000 loss: 0.0387 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 5/5, Iter: 51/93 -- label: 4.0000 loss: 0.0483 
INFO:ignit

## Check the prediction on the test dataset

In [9]:
dataset_dir = Path(root_dir, "MedNIST")
class_names = sorted(f"{x.name}" for x in dataset_dir.iterdir() if x.is_dir())
testdata = MedNISTDataset(root_dir=root_dir, transform=transform, section="test", download=False, runtime_cache=True)

max_items_to_print = 10
with eval_mode(model):
    for item in DataLoader(testdata, batch_size=1, num_workers=0):
        prob = np.array(model(item["image"].to("cuda:0")).detach().to("cpu"))[0]
        pred = class_names[prob.argmax()]
        gt = item["class_name"][0]
        print(f"Class prediction is {pred}. Ground-truth: {gt}")
        max_items_to_print -= 1
        if max_items_to_print == 0:
            break

Class prediction is AbdomenCT. Ground-truth: AbdomenCT
Class prediction is BreastMRI. Ground-truth: BreastMRI
Class prediction is ChestCT. Ground-truth: ChestCT
Class prediction is CXR. Ground-truth: CXR
Class prediction is Hand. Ground-truth: Hand
Class prediction is HeadCT. Ground-truth: HeadCT
Class prediction is HeadCT. Ground-truth: HeadCT
Class prediction is CXR. Ground-truth: CXR
Class prediction is ChestCT. Ground-truth: ChestCT
Class prediction is BreastMRI. Ground-truth: BreastMRI
